Part 2 of **time series forecasting with energy**

In this section, I will investigate the problem at hand, formalising it, outlining methods for interpreting error, and setup a baseline using standard statistical methods for time-series analysis. The library `statsmodels` will come in handy with this task.

In [0]:
# LOAD THE REPOSITORY
# if you are working from outside the repository
# this happens if you use colab like I do
!git clone https://github.com/sandeshbhatjr/energy-prediction.git
!pip install -U --quiet pandas statsmodels tables numpy

In [0]:
import datetime as dt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

df = pd.read_hdf('energy-prediction/data/clean_german_df', key='df_with_load_and_gen')

If the read_hdf throws an error, then it's probably because the version of pyTables, or numpy is the wrong one. This is a known bug in some combination of versions of those libraries: see https://stackoverflow.com/questions/54210073/pd-read-hdf-throws-cannot-set-writable-flag-to-true-of-this-array for details.

# Statistical models

This part will be technical. The purpose is to explore SOTA models for time-series forecasting, and their applicability to day-ahead price forecasting. In a typical use-case, we have access to a history of recorded day-ahead prices, and we are interested in forecasting the day-ahead prices in an interval consisting of a tuple of 24 prices for each day. The problem can be formulated precisely as follows.
>  Let $\{t_1, t_2, t_3, ..., t_k \} := \mathfrak{T}$ represent a collection of dates in  chronological order, $p_t \in \mathbb{R}^{24}$ be the day-ahead price indexed by $t \in \mathfrak{T}$. Given some historic day-ahead prices $p_{t_1}, p_{t_2}, ... ,p_{t_k}$, forecast the future day-ahead prices $p_{t_{k+1}}, ..., p_{t_{k+h}}$ in a certain future window-size of $h \in \mathbb{N}$. In addition, we have a collection of exogenous time series- $d^{\alpha}_{t_1},d^{\alpha}_{t_2}, ..., d^{\alpha}_{t_k}$ indexed by $\alpha$, which includes the generation and consumption forecasts. Finally for each $t\in {t_1, ..., t_{k+h'}}, h'>> h$, we have some associated features $f^{\beta}_t$, where the features are known well into future (for example, if the day is a holiday or not).

In [24]:
def convert_to_multivariate_series(df, data_columns):
  return df[data_columns].groupby(by=[df.index.date, df.index.time]).mean().unstack(-1)

convert_to_multivariate_series(df, ['Day Ahead Price', 'Wind offshore Generation'])

Day Ahead Price                    ... Wind offshore Generation                    
                  00:00:00 01:00:00 02:00:00  ...                 21:00:00   22:00:00 23:00:00
2015-01-01             NaN      NaN      NaN  ...                  66.7500    67.0000    67.00
2015-01-02             NaN      NaN      NaN  ...                 158.0000   159.0000   158.25
2015-01-03             NaN      NaN      NaN  ...                 155.0000   154.7500   153.75
2015-01-04             NaN      NaN      NaN  ...                      NaN        NaN      NaN
2015-01-05             NaN      NaN      NaN  ...                  61.9375    63.9375    57.75
...                    ...      ...      ...  ...                      ...        ...      ...
2020-02-02           -4.97   -10.10   -16.95  ...                 496.0000   586.7500   726.25
2020-02-03           15.92    15.55    14.38  ...                1366.7500  1393.0000  1393.50
2020-02-04           20.79    17.41    16.24  ...                 996.0000   916.0000   797.25
2020-02-05           25.44    25.00    24.43  ...                1048.5000  1060.2500  1061.75
2020-02-06           26.51    25.35    24.47  ...                 316.2500   260.5000   201.75

[1863 rows x 48 columns]

**Windowed dataset using Keras Timeseries generator**

Most models will work not on the entire time-series, but on a windowed subset of the time series. Keras has an inbuilt windowed dataset generator, but it helps to know a few things before we use that. It is primarily meant for use with Keras, so it produces a generator that outputs the dataset in batches as a list $[b_1, b_2, ...]$. Each batch $b_i$ consists of $[X,y]$, with each being a numpy array. The format here is as follows: each $X$ is a windowed dataset with following indices: `(sample_num, timestep, feature)`, while $y$ is of the form `(sample_num, value)`. The assumption here is that `y` consists of a single timestep, which works fine in our case.

In [0]:
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

def get_windowed_dataset(df, X_cols, y_cols, window_size, return_generator=False, **kwargs):
  X_data = extractHourlyData(df, X_cols)
  y_data = extractHourlyData(df, y_cols)
  # use batch size if defined, else return the whole dataset in one batch
  batch_size = kwargs['batch_size'] or len(X_data)
  generator = TimeseriesGenerator(
    X_data, 
    y_data, 
    length = window_size, 
    sampling_rate = 1,
    batch_size = batch_size)
  if return_generator:
    return generator
  else:
    X_batches = [X for (X, y) in generator]
    y_batches = [y for (X, y) in generator]
    return X_batches, y_batches

Just a quick check to see if the generator works as expected:

In [10]:
generator = get_windowed_dataset(
    df, 
    ['Day Ahead Price'], 
    ['Day Ahead Price'], 
    1, 
    return_generator=True, 
    batch_size=len(df)
)

generator[0][0].shape, generator[0][1].shape

KeyError: ignored

Before proceeding to investigate our models, it is important to figure out a cross-validation strategy and testing strategy using a metric to assess the performance of each of our models. So, let us briefly touch upon these topics first.  

1. The cross-validation strategy for temporal data is a bit different from randomly choosing a subset of the dataset. Since the goal of our analysis is to forecast future values using present data, we will likewise train on a dataset from the past, and validate it against a subset to the future of it. This simulates the actual situation at hand; see [3, 4] for more details.
2. For a simple criterion to do a quick comparison of algorithms as a first step in model evaluation, the sMAPE suffices:
$$s = \frac{|y-\hat{y}|}{2(|y| + |\hat{y}|)} $$ This should be sufficient for a first-order estimate of how well our models are doing; later on, we can perform more sophisticated forms of tests for our analysis.

To cover pt. 1, we create a function to extract dataset into a train and test form chronologically separated as described. We also create a GridsearchCV equivalent for time series for hyperparameter search in ML models.

In [0]:
def test_train_timesplit(df, date_col_name='Date', train_size=0.9, test_size=0.1):
  """
    Returns test-train split data based on date with test chronologically later than train data.
  """
  min_date = df[date_col_name].min()
  max_date = df[date_col_name].max()
  train_split_date = min_date + (train_size*(max_date - min_date))
  test_split_date = train_split_date + (test_size*(max_date - min_date))
  train_df = df[df[date_col_name] < train_split_date]
  test_df = df[(df[date_col_name] > train_split_date) & (df[date_col_name] < test_split_date)]
  return train_df, test_df

def day_forward_chaining(df, date_col_name='Date', k=10):
  for i in range(1,k):
    yield test_train_timesplit(df, date_col_name, train_size=(i/k), test_size=(1/k))

# The above function is the time-series equivalent of gridsearch CV
# Example use-case:
# for train_df, test_df in day_forward_chaining(german_df):
#   <<< do your model training and tuning here >>>

For the second part, to compute sMAPE for model evaluation, the following class is defined.

In [0]:
class error_analysis:
  def __init__(self, predictor):
    self.predictor = predictor
  def smape_by_entry_from_generator(self, generator):
    smape_error = {}
    yhat, y = np.zeros((0,24)), np.zeros((0,24))
    for X_i, y_i in generator:
      yhat_i = self.predictor(X_i)
      assert yhat_i.shape == y_i.shape
      # add to our list
      yhat, y = np.concatenate([yhat_i, yhat], axis=0), np.concatenate([y_i, y], axis=0)
    for h in range(24):
      smape_error[h] = np.absolute(yhat[:,h] - y[:,h])/(np.absolute(yhat[:,h]) + np.absolute(y[:,h]))
    return pd.DataFrame(smape_error)
  def smape_by_entry_from_dataset(self, X, y):
    smape_error = {}
    yhat, y = np.zeros((0,24)), np.zeros((0,24))
    for X_i, y_i in generator:
      yhat_i = self.predictor(X_i)
      assert yhat_i.shape == y_i.shape
      # add to our list
      yhat, y = np.concatenate([yhat_i, yhat], axis=0), np.concatenate([y_i, y], axis=0)
    for h in range(24):
      smape_error[h] = np.absolute(yhat[:,h] - y[:,h])/(np.absolute(yhat[:,h]) + np.absolute(y[:,h]))
    return pd.DataFrame(smape_error)    
  def smape_by_hour(self, generator):
    error_df = self.smape_by_entry_from_generator(generator)
    error_df.replace(np.inf, 0) # remove infinite values
    return error_df.mean(axis=0, skipna=True)*200
  def total_smape(self, generator):
    smape_by_hour = self.smape_by_hour(generator)
    return smape_by_hour.mean(axis=0, skipna=True)
  def plot(self, generator):
    plt.figure(figsize=(15,8))
    error_df = self.smape_by_entry(generator)
    error_df.boxplot()

# Naive model and the baseline

The simplest model one can think of is as follows: $p_{t+h} = p_t $ for all $h$ in the future window. This should allow us to setup a baseline, which any sensible model should perform better than. This is primarily helpful in debugging complex models when making sure that we are not doing anything terribly stupid.

In [0]:
def naivemodel(X):
  return X[:,0,:]

# Since this model needs no training, 
# we can just test it on the whole dataset
X, y = get_windowed_dataset(german_df, ['Day Ahead Price'], ['Day Ahead Price'], 1)

def val_generator(): 
  yield [X, y]

naive_error = error_analysis(naivemodel)
naive_error.total_smape(val_generator())

So, there's our baseline. Anything that performs worse usually means that we are doing something wrong.  

Another simple baseline is to simply take the weighted average of the previous values.

In [0]:
def mamodel(X, alpha=0.1):
  window_size = X.shape[1]
  weight = np.array([pow(alpha, i) for i in reversed(range(window_size))])
  return np.tensordot(weight, X, axes=(0,1)) / np.sum(weight)

# Regression

For time-series data, regression usually presents itself in the form of auto-regression, i.e. regression in terms of previous time-step variable. A very simple example would be the $AR(k)$ models, where the hypothesis is as follows:
$$p_{t+h}=\sum_{i=0}\alpha_ip_{t-i}+\epsilon$$ where $\epsilon$ is random white noise.

The above technique is a very simple one; there are far more sophistical statistical techniques available. We will pursue them here through the use of `statsmodels` library. I will not explore the algorithms themselves in detail (though it will make for a good series of blog posts sometime in the near future; for now refer to [1] for details).

## Univariate AR(k) with time trend

A simple way to approach the problem is to consider each hourly day-ahead price as a separate time-series. This lets us use the AR(k) and ARMA(p,q) models on them directly.

In [0]:
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics

AR_model = AutoReg(german_df, 3, trend='t')

In [0]:
sns.set_style('darkgrid')
sns.mpl.rc('figure',figsize=(16, 6))
res.plot_predict()

## Multivariate: VAR(k) and VECM

Instead of treating our series as a univariate time series for each hour, one can expect an improvement by considering it proper as a multivariate time series analysis. The relevant algorithms are the vector autoregressive model (VAR) and the vector error correction model (VECM).

# Exponential Smoothing and Holtz-Winter models

Another popular approach to time-series problems are the ETS class of techniques.

In [0]:
from statsmodels.tsa.api import SimpleExpSmoothing

$$y_t = y_{t-1} + \alpha (y_{t-1} - \hat{y}_{t-1})$$

$$y_{t+h} = l_t +h b_t + s_{t+h-m(k+1)}$$
$$l_t = \alpha \frac{y_t}{s_{t-m}} + (1-\alpha)(l_{t-1} + b_{t-1})$$
$$b_t = \beta^*(l_t - l_{t-1}) + (1-\beta^*)b_{t-1}$$
$$s_t = \gamma \frac{y_t}{(l_{t-1} + b_{t-1})} + (1-\gamma)s_{t-m}$$

In [0]:
from statsmodels.tsa.api import ExponentialSmoothing, Holt

# Markov switching regression

# Bibliography

**[HA18]** R. J. Hyndman, and G. Athanasopoulos, Forecasting: Principles and Practice, https://otexts.com/fpp2/index.html 

**[HWL15]**. R. J. Hyndman, E. Wang, N. Laptev, *Large-Scale Unusual Time Series Detection*,  [https://robjhyndman.com/papers/icdm2015.pdf]

**[SE11]** More on the different approaches to the validation strategies for a time series here: https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection

**[Tas00]** L. J. Tashman, *Out-of-sample tests of forecasting accuracy: an analysis and review*, International Journal of Forecasting, 2000, vol. 16, issue 4, 437-450 [https://econpapers.repec.org/article/eeeintfor/v_3a16_3ay_3a2000_3ai_3a4_3ap_3a437-450.htm].

Supplements:  
[Using Facebook's forecast engine: Prophet]()  
[GluonTS]()

Next part: [Deep Models]()